In [4]:
# Setup sys.path to import from src/
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [15]:
# Spark session
from pyspark.sql import SparkSession, functions as F

# Import custom functions
from src.preprocessing import load_data, clean_data, feature_engineer, assemble_features
from src.save_outputs import save_predictions, save_feature_importances, save_model_metadata
from src.model import train_random_forest, pick_best_model_from_grid, evaluate_model, manual_grid_search_rf


In [6]:
spark = SparkSession.builder \
    .appName("Ecommerce Behavior Exploration") \
    .master("local[*]") \
    .getOrCreate()


In [7]:
# Load raw dataset
file_path = "../data/testData-2019-Nov.csv"
raw_df = load_data(file_path, spark)

# Quick look at raw data
raw_df.show(5)
raw_df.printSchema()


+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-19 08:35:...|      view|  30200005|2053013554449088861|                NULL|   elari|  77.2|512412397|f62be3c5-18af-4ab...|
|2019-11-26 14:16:...|      view|   1005115|2053013555631882655|electronics.smart...|   apple|916.37|568675496|c857db53-cd0a-480...|
|2019-11-10 17:50:...|      view|  15700275|2053013559733912211|                NULL|imperial|206.16|513262731|c637d18a-6fc5-4c1...|
|2019-11-04 14:23:...|      view|   1004589|2053013555631882655|electronics.smart...|    inoi| 61.36|562973725|e41d3c3f-830e-48d...|
|2019-11-29 17:11:...|  purchase|   5300157|2053013563173241677|     

In [9]:
# Clean the dataset
clean_df = clean_data(raw_df)

# Quick check after cleaning
clean_df.show(5)


+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-19 03:35:46|      view|  30200005|2053013554449088861|                NULL|   elari|  77.2|512412397|f62be3c5-18af-4ab...|
|2019-11-26 09:16:08|      view|   1005115|2053013555631882655|electronics.smart...|   apple|916.37|568675496|c857db53-cd0a-480...|
|2019-11-10 12:50:50|      view|  15700275|2053013559733912211|                NULL|imperial|206.16|513262731|c637d18a-6fc5-4c1...|
|2019-11-04 09:23:52|      view|   1004589|2053013555631882655|electronics.smart...|    inoi| 61.36|562973725|e41d3c3f-830e-48d...|
|2019-11-29 12:11:17|  purchase|   5300157|2053013563173241677|             

In [10]:
# Create new features if needed
feature_df = feature_engineer(clean_df)

# Quick preview
feature_df.show(5)


+--------------------+---------+-------------+-------------+-------------------+-------------------+---------+----------------+
|        user_session|num_views|num_cart_adds|num_purchases|      session_start|        session_end|avg_price|session_duration|
+--------------------+---------+-------------+-------------+-------------------+-------------------+---------+----------------+
|08327d25-5fe5-4ec...|        1|            0|            0|2019-11-10 23:43:37|2019-11-10 23:43:37|     39.9|               0|
|cbd7e3a0-2c5e-493...|        1|            0|            0|2019-11-17 15:48:48|2019-11-17 15:48:48|     4.61|               0|
|a08f39b4-71f4-431...|        1|            0|            0|2019-11-24 10:21:36|2019-11-24 10:21:36|    23.81|               0|
|544365e8-d7e6-4ea...|        1|            1|            0|2019-11-09 11:44:13|2019-11-09 11:46:48|    47.47|             155|
|71697745-4b20-419...|        1|            0|            0|2019-11-17 12:22:24|2019-11-17 12:22:24|   1

In [11]:
labels = clean_df.groupBy("user_session").agg(
    (F.max(F.when(F.col("event_type") == "purchase", 1).otherwise(0))).alias("label")
)

final_df = feature_df.join(labels, on="user_session", how="left").fillna(0)


In [12]:
# Feature columns you want to use
feature_cols = ['num_views', 'num_cart_adds', 'session_duration', 'avg_price']

# Assemble features
final_df = assemble_features(final_df, feature_cols)

In [13]:
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)

In [16]:
manual_grid_search_rf(train_df, test_df)

🚀 Starting Manual Random Forest grid search with model saves...
🔵 Training: numTrees=50, maxDepth=5, maxBins=32
🔹 AUC: 0.9969


d:\ecomm-bigdata-project\src\model.py:117: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


✅ Predictions saved: D:/ecomm-bigdata-project/output/rf/rf_50_5_32\rf_50_5_32_predictions.csv
✅ Feature importances saved: D:/ecomm-bigdata-project/output/rf/rf_50_5_32\rf_50_5_32_feature_importances.csv
✅ Model metadata saved: D:/ecomm-bigdata-project/output/rf/rf_50_5_32\rf_50_5_32_metadata.json
🔵 Training: numTrees=100, maxDepth=5, maxBins=32
🔹 AUC: 0.9972
✅ Predictions saved: D:/ecomm-bigdata-project/output/rf/rf_100_5_32\rf_100_5_32_predictions.csv
✅ Feature importances saved: D:/ecomm-bigdata-project/output/rf/rf_100_5_32\rf_100_5_32_feature_importances.csv
✅ Model metadata saved: D:/ecomm-bigdata-project/output/rf/rf_100_5_32\rf_100_5_32_metadata.json
🔵 Training: numTrees=200, maxDepth=5, maxBins=32


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\amrut\AppData\Local\Programs\Python\Python310\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\amrut\AppData\Local\Programs\Python\Python310\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\Users\amrut\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
base_rf_dir = "D:/ecomm-bigdata-project/output/rf/"
results_csv_path = os.path.join(base_rf_dir, "grid_search_results.csv")

best_model_outputs = pick_best_model_from_grid(results_csv_path, base_rf_dir)